In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import cv2
import numpy as np
import pandas as pd
import os

In [2]:
# === USER PATHS ===

In [3]:
IMAGE1_PATH = r"C:\Users\ragha\Downloads\IMAGE1.tif"
IMAGE2_PATH = r"C:\Users\ragha\Downloads\IMAGE2.tif"

In [4]:
# === Utility: Convert OpenCV → Tkinter image ===

In [5]:
def cv2_to_tkimg(cv_img, max_size=None):
    rgb = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(rgb)
    if max_size:
        if hasattr(Image, "Resampling"):
            pil.thumbnail(max_size, Image.Resampling.LANCZOS)
        else:
            pil.thumbnail(max_size, Image.ANTIALIAS)
    return ImageTk.PhotoImage(pil)


class ImageAlignApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Alignment Studio — Save Full Transform CSV")
        self.root.attributes("-fullscreen", True)
        self.image1 = cv2.imread(IMAGE1_PATH)
        self.image2 = cv2.imread(IMAGE2_PATH)
        if self.image1 is None or self.image2 is None:
            messagebox.showerror("Error", "❌ Could not load one or both images.")
            root.destroy()
            return
        self.params = {
            "rotation": 0,
            "scale": 1.0,
            "offset_x": 0,
            "offset_y": 0,
            "transparency": 50,
            "warp_tl_x": 0, "warp_tl_y": 0,
            "warp_tr_x": 0, "warp_tr_y": 0,
            "warp_bl_x": 0, "warp_bl_y": 0,
            "warp_br_x": 0, "warp_br_y": 0,
            "diag1": 0, "diag2": 0,
            "hstretch": 0, "vstretch": 0,
            "top_tilt_x": 0, "top_tilt_y": 0,
            "bottom_tilt_x": 0, "bottom_tilt_y": 0,
        }

        self.drag_data = {"x": 0, "y": 0}
        self.create_widgets()
        self.bind_events()
        self.root.bind("<Configure>", lambda e: self.update_preview_image())
        self.update_preview_image()

    def create_widgets(self):
        control_frame = tk.Frame(self.root, bg="#111", height=140)
        control_frame.pack(side=tk.TOP, fill=tk.X)

        tk.Button(control_frame, text="💾 Save & Export CSV", command=self.save_all,
                  font=("Arial", 10, "bold"), bg="#28a745", fg="white", padx=10, pady=5)\
            .pack(side=tk.RIGHT, padx=10, pady=10)

        tk.Button(control_frame, text="📂 Load CSV", command=self.load_csv,
                  font=("Arial", 10, "bold"), bg="#007bff", fg="white", padx=10, pady=5)\
            .pack(side=tk.RIGHT, padx=10, pady=10)

        tk.Button(control_frame, text="❌ Exit", command=self.root.destroy,
                  font=("Arial", 10, "bold"), bg="#d9534f", fg="white", padx=10, pady=5)\
            .pack(side=tk.RIGHT, padx=10, pady=10)

        sliders = [
            ("Rotation (°)", "rotation", -180, 180),
            ("Zoom", "scale", 0.2, 3.0),
            ("Offset X", "offset_x", -500, 500),
            ("Offset Y", "offset_y", -500, 500),
            ("Transparency", "transparency", 0, 100),
        ]
        for label, key, minv, maxv in sliders:
            frame = tk.Frame(control_frame, bg="#111")
            frame.pack(side=tk.LEFT, padx=10)
            tk.Label(frame, text=label, fg="white", bg="#111").pack()
            ttk.Scale(frame, from_=minv, to=maxv, value=self.params[key],
                      orient="horizontal", command=lambda val, k=key: self.update_param(k, val)).pack()

        self.add_section("Corner Warp", "#222", [
            ("TL-X", "warp_tl_x"), ("TL-Y", "warp_tl_y"),
            ("TR-X", "warp_tr_x"), ("TR-Y", "warp_tr_y"),
            ("BL-X", "warp_bl_x"), ("BL-Y", "warp_bl_y"),
            ("BR-X", "warp_br_x"), ("BR-Y", "warp_br_y"),
        ])

        self.add_section("Diagonal Stretch", "#333", [
            ("Diagonal ↘ (TL↔BR)", "diag1"),
            ("Diagonal ↙ (TR↔BL)", "diag2"),
        ], (-150, 150))

        self.add_section("H & V Stretch", "#444", [
            ("Horizontal (←→)", "hstretch"),
            ("Vertical (↑↓)", "vstretch"),
        ], (-150, 150))

        self.add_section("Top & Bottom Tilt", "#555", [
            ("Top Tilt X", "top_tilt_x"), ("Top Tilt Y", "top_tilt_y"),
            ("Bottom Tilt X", "bottom_tilt_x"), ("Bottom Tilt Y", "bottom_tilt_y"),
        ], (-200, 200))

        self.canvas = tk.Canvas(self.root, bg="black")
        self.canvas.pack(fill=tk.BOTH, expand=True)

    def add_section(self, title, color, controls, limits=(-200, 200)):
        frame = tk.LabelFrame(self.root, text=title, fg="white", bg=color)
        frame.pack(side=tk.BOTTOM, fill=tk.X, padx=10, pady=5)
        for label, key in controls:
            sub = tk.Frame(frame, bg=color)
            sub.pack(side=tk.LEFT, padx=5)
            tk.Label(sub, text=label, fg="white", bg=color).pack()
            ttk.Scale(sub, from_=limits[0], to=limits[1], value=self.params[key],
                      orient="horizontal", command=lambda val, k=key: self.update_param(k, val)).pack()

    def bind_events(self):
        self.canvas.bind("<ButtonPress-1>", self.start_drag)
        self.canvas.bind("<B1-Motion>", self.do_drag)
        self.canvas.bind("<MouseWheel>", self.do_zoom)
        self.root.bind("<Escape>", lambda e: self.root.destroy())

    def start_drag(self, event):
        self.drag_data["x"] = event.x
        self.drag_data["y"] = event.y

    def do_drag(self, event):
        dx = event.x - self.drag_data["x"]
        dy = event.y - self.drag_data["y"]
        self.params["offset_x"] += dx
        self.params["offset_y"] += dy
        self.drag_data["x"] = event.x
        self.drag_data["y"] = event.y
        self.update_preview_image()

    def do_zoom(self, event):
        if event.delta > 0:
            self.params["scale"] *= 1.05
        else:
            self.params["scale"] *= 0.95
        self.update_preview_image()

    def update_param(self, key, val):
        self.params[key] = float(val)
        self.update_preview_image()

    def apply_transformations(self, img):
        rows, cols = img.shape[:2]
        center = (cols // 2, rows // 2)

        M_rot = cv2.getRotationMatrix2D(center, self.params["rotation"], self.params["scale"])
        img = cv2.warpAffine(img, M_rot, (cols, rows))

        pts1 = np.float32([[0, 0], [cols, 0], [0, rows], [cols, rows]])
        tl = [self.params["warp_tl_x"], self.params["warp_tl_y"]]
        tr = [self.params["warp_tr_x"], self.params["warp_tr_y"]]
        bl = [self.params["warp_bl_x"], self.params["warp_bl_y"]]
        br = [self.params["warp_br_x"], self.params["warp_br_y"]]
        d1, d2, h, v = self.params["diag1"], self.params["diag2"], self.params["hstretch"], self.params["vstretch"]
        top_x, top_y, bottom_x, bottom_y = self.params["top_tilt_x"], self.params["top_tilt_y"], \
                                           self.params["bottom_tilt_x"], self.params["bottom_tilt_y"]

        tl[0] -= d1 + h - top_x; tl[1] -= d1 + v - top_y
        tr[0] += d2 + h + top_x; tr[1] -= d2 + v - top_y
        bl[0] -= d2 + h - bottom_x; bl[1] += d2 + v + bottom_y
        br[0] += d1 + h + bottom_x; br[1] += d1 + v + bottom_y

        pts2 = np.float32([
            [0 + tl[0], 0 + tl[1]],
            [cols + tr[0], 0 + tr[1]],
            [0 + bl[0], rows + bl[1]],
            [cols + br[0], rows + br[1]],
        ])
        matrix = cv2.getPerspectiveTransform(pts1, pts2)
        return cv2.warpPerspective(img, matrix, (cols, rows))

    def update_preview_image(self):
        if self.canvas.winfo_width() < 10:
            return
        img1 = self.image1.copy()
        img2 = self.apply_transformations(self.image2)
        alpha = self.params["transparency"] / 100.0

        M_offset = np.float32([[1, 0, self.params["offset_x"]],
                               [0, 1, self.params["offset_y"]]])
        img2 = cv2.warpAffine(img2, M_offset, (img1.shape[1], img1.shape[0]))

        blended = cv2.addWeighted(img1, 1, img2, alpha, 0)
        tkimg = cv2_to_tkimg(blended, max_size=(self.canvas.winfo_width(), self.canvas.winfo_height()))
        self._tkimg = tkimg
        self.canvas.delete("all")
        self.canvas.create_image(self.canvas.winfo_width() // 2,
                                 self.canvas.winfo_height() // 2, image=tkimg)

    def save_all(self):
        path = filedialog.asksaveasfilename(defaultextension=".png",
                                            filetypes=[("PNG Files", "*.png")],
                                            title="Save Aligned Image")
        if not path:
            return

        transformed = self.apply_transformations(self.image2)
        M_offset = np.float32([[1, 0, self.params["offset_x"]],
                               [0, 1, self.params["offset_y"]]])
        transformed = cv2.warpAffine(transformed, M_offset,
                                     (self.image1.shape[1], self.image1.shape[0]))
        alpha = self.params["transparency"] / 100.0
        final = cv2.addWeighted(self.image1, 1, transformed, alpha, 0)

        cv2.imwrite(path, final)
        csv_path = os.path.splitext(path)[0] + "_transform.csv"
        pd.DataFrame([self.params]).to_csv(csv_path, index=False)

        messagebox.showinfo("Saved", f"Image saved:\n{path}\n\nParameters CSV:\n{csv_path}")

    def load_csv(self):
        csv_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")],
                                              title="Select Transform CSV")
        if not csv_path:
            return

        try:
            data = pd.read_csv(csv_path).iloc[0].to_dict()
            self.params.update({k: float(v) for k, v in data.items() if k in self.params})
            self.update_preview_image()
            messagebox.showinfo("Loaded", f"Transform parameters loaded from:\n{csv_path}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load CSV:\n{e}")

In [6]:
# === RUN ===

In [7]:
root = tk.Tk()
app = ImageAlignApp(root)
root.mainloop()